# Exercise 4.3. 
Implement VI in Matlab and compute the optimal policy
and optimal value function for the problem described in Exercise 3.3. Com-
pare the results obtained through VI with the theoretical result obtained in
Exercise3.3 Compare results obtaiued by VI and PI.
![img](imgs/Screenshot_2018-04-20_11-23-41.png)

In [1]:
import numpy as np
import itertools as IT
np.set_printoptions(precision=2, suppress=True)
#initial parameters
gamma = .9
R = np.matrix([[-1],[.6],[.5],[-.9]])
P = np.matrix([[0.8,0.2],[0.2,0.8],[0.3,0.7],[0.9,0.1]])

N_states = 2
N_actions = 2
N_steps = 400
N_steps_pe = 20 
N_steps_pi = 50

In [2]:
from numpy.linalg import inv
import itertools as IT


# function to evaluate the state value(V) function of a certain policy
def eval_v(policy):
    P_pi = np.matmul(policy, P)
    R_pi = np.matmul(policy, R)

    return np.matmul(inv(np.identity(P_pi.shape[0]) - gamma * P_pi), R_pi)


# function to evaluate the state-action(Q) value function of a certain policy
def eval_q(policy):
    # product of the transition matrix with the policy
    P_aux = np.matmul(P, policy)

    return np.matmul(inv(np.identity(P_aux.shape[0]) - gamma * P_aux), R)


# rewards for a state
def R_state(state):
    return R[N_actions * state: N_actions * state + N_actions]


# transition probabilities for a state
def P_state(state):
    return P[N_actions * state: N_actions * state + N_actions, ]


def R_(state, action=None):
    if action == None:
        return R_state(state)
    else:
        return R_state(state)[action]


def P_(state, action=None, state_t1=None):
    if action == None:
        if state_t1 == None:
            return P_state(state)
        else:
            return P_state(state)[:, state_t1]
    else:
        if state_t1 == None:
            return P_state(state)[action,]
        else:
            return P_state(state)[action, state_t1]


def policy_(policy, state, action=None):
    if action is not None:
        return policy[state][state * N_actions + action]
    else:
        return [policy[state][state * N_actions + action_] for action_ in range(N_actions)]
        

# We define the algorithm for policy evaluation with the state value function
def policy_evaluation_v(policy):
    # 1
    v = np.zeros(N_states)
    # 2, 8 stop condition is not stated, instead we do 400 iterations
    for i in range(N_steps):
        # 3
        delta = 0
        # 4
        for state in range(N_states):
            # 5
            v_old = v[state]
            # 6
            v_aux = 0
            for action in range(N_actions):
                v_aux += policy_(policy, state, action) * (R_(state, action) + gamma * sum(
                    [P_(state, action, state_t1) * v[state_t1] for state_t1 in range(N_states)]))
            v[state] = v_aux
            # 7
            # delta = np.max(delta, np.abs(v_old - v[state]))

    # 8,9
    return v


# We define the algorithm for policy evaluation with the state-action
# value function
# We define the algorithm for policy evaluation with the state value function
def policy_evaluation_q(policy):
    # 1
    q = np.zeros(N_states * N_actions)
    # 2, 8 stop condition is not stated, instead we do 400 iterations
    for i in range(N_steps):
        # 3
        delta = 0
        # 4 TO-DO
        for state, action in IT.product(range(N_states), range(N_actions)):
            # 5
            q_old = q[state * N_actions + action]
            # 6
            q_aux = [R_(state, action)]
            for state_t1 in range(N_states):
                q_aux += gamma * P_(state, action, state_t1) * sum(
                    [policy_(policy, state_t1, action_t1) * q[state_t1 * N_actions + action_t1] for action_t1 in
                     range(N_actions)])
            q[state * N_actions + action] = q_aux
            # 7
            # delta = np.max(delta, np.abs(q_old - q[state,action]))

    return q

# Policy iteration for state value function
def policy_iteration_v(policy):
    v = np.zeros(N_states) #1
    theta = False
    while not theta: #2,3
        v = policy_evaluation_v(policy) # 4-9
        #print('\n v:',v)
        theta = True #10
        for s in range(N_states): #11
            #print('\n s:',s)
            a = policy_(policy, s) #12
            #print('a:',a)  
            
            #13
            arg_max = np.argmax([(R_(s, a_t1) + gamma *
                                 np.sum([np.dot(P_(s, a_t1, s_t1), v[s_t1]) for s_t1 in range(N_states)]))
                                 for a_t1 in range(N_actions)])
            #print('arg_max:',arg_max)
            policy[s] = [0] * len(policy[s])
            policy[s][N_actions * s + arg_max] = 1
            
            #print('pi[s]:',policy[s])
            #print('policy_(policy, s):',policy_(policy, s))
            if not (a == policy_(policy, s)): theta = False #14
                #print('a not equal to policy_(policy, s)')
                
            #print('theta:',theta)
        #print('pi:',policy)
    return policy #15

# Policy iteration for state-action value function
def policy_iteration_q(policy):
    q = np.zeros(N_states * N_actions) #1
    theta = False
    while not theta: #2,3
        q = policy_evaluation_q(policy) # 4-9
        #print('\n q:',q)
        theta = True #10
        for s in range(N_states): #11
            #print('\n s:',s)
            a = policy_(policy, s) #12
            #print('a:',a)  
            
            #13
            arg_max = np.argmax([q[s * N_actions + a_t1]  for a_t1 in range(N_actions)])
            #print('arg_max:',arg_max)
            policy[s] = [0] * len(policy[s])
            policy[s][N_actions * s + arg_max] = 1
            
            #print('pi[s]:',policy[s])
            #print('policy_(policy, s):',policy_(policy, s))
            if not (a == policy_(policy, s)): theta = False #14
                #print('a not equal to policy_(policy, s)')
                
            #print('theta:',theta)
        #print('pi:',policy)
    return policy #15


## Value Iteration for V functions
![img](imgs/4.3.1.png)

In [3]:
def value_iteration_v(debug=False):
    if debug : print('Value Iteration with V:')
    policy = np.zeros([N_states,(N_states*N_actions)]) 
    v = np.zeros(N_states) #1
    epsilon = 0.01
    delta = 10000000
    while delta>epsilon: #2
        delta = 0 #3
        
        if debug : print('\n v:',v)
        for s in range(N_states): #4
            if debug : print('\n s:',s)
            
            v_old = v[s] #5
            if debug : print('\n v_old:',v_old)
            #6
            v[s]=np.max([(R_(s, a) + gamma *
                         np.sum([np.dot(P_(s, a, s_t1), v[s_t1]) for s_t1 in range(N_states)]))
                         for a in range(N_actions)])
            if debug : print('\n v[s]:',v[s])
            delta = max(delta, np.abs(v_old-v[s]))#8
            if debug : print('delta:',delta)

    # (out of the while) 
    if debug : print('\n For each state: \n')
    for s in range(N_states): #9
        #10
        arg_max = np.argmax([(R_(s, a) + gamma *
                             np.sum([np.dot(P_(s, a, s_t1), v[s_t1]) for s_t1 in range(N_states)]))
                             for a in range(N_actions)])
        if debug : print('arg_max:',arg_max)
        policy[s] = [0] * (N_states*N_actions)
        policy[s][N_actions * s + arg_max] = 1
            
        if debug : print('pi[s]:',policy[s])


    if debug : print('pi:',policy)
    return policy #11

*- To prove that they work a will show a print step by step a few cells below*

In [4]:
print('\n value iteration with v: \n',np.array(value_iteration_v()))


 value iteration with v: 
 [[0. 1. 0. 0.]
 [0. 0. 1. 0.]]


## Value Iteration for Q functions
![img](imgs/4.3.2.png)

In [5]:
def value_iteration_q(debug=False):
    if debug : print('Value Iteration with Q:')
    policy = np.zeros([N_states,(N_states*N_actions)]) 
    q = np.zeros(N_states*N_actions) #1
    epsilon = 0.01
    delta = 10000000
    while delta>epsilon: # 2, 8
        delta = 0 #3
        
        if debug : print('\n q:',q)
        if debug : print('\n For each state action pair: \n:')    
        for s , a in IT.product(range(N_states),range(N_actions)): #4
            if debug : print('\n s , a:', s , a)
            
            q_old = q[s * N_actions + a] #5
            if debug : print('\n q_old:',q_old)
            
            #6
            q[s * N_actions + a] = R_(s, a) + gamma * np.sum([np.dot(P_(s, a, s_t1), 
                                                    np.max([q[s_t1 * N_actions + a_t1] for a_t1 in range(N_actions)])) 
                                              for s_t1 in range(N_states)])
           
         
            if debug : print('\n q(s,a)',q[s * N_actions + a])
            delta = max(delta, np.abs(q_old-q[s * N_actions + a]))#7
            if debug : print('delta:',delta)

    # (out of the while) 
    if debug : print('\n For each state: \n')
    for s in range(N_states): #9
        #10
        arg_max = np.argmax([q[s * N_actions + a_t1]  for a_t1 in range(N_actions)])
        if debug : print('arg_max:',arg_max)
        policy[s] = [0] * (N_states*N_actions)
        policy[s][N_actions * s + arg_max] = 1
            
        if debug : print('pi[s]:',policy[s])

    if debug : print('pi:',policy)
    return policy #11

*- To prove that they work a will show a print step by step a few cells below*

In [6]:
print('\n value iteration with q: \n',np.array(value_iteration_q()))


 value iteration with q: 
 [[0. 1. 0. 0.]
 [0. 0. 1. 0.]]


## Step by Step Value Iteration with V

In [ ]:
print('\n value iteration with q: \n',np.array(value_iteration_v(True)))

## Step by Step Value Iteration with Q

In [ ]:
print('\n value iteration with q: \n',np.array(value_iteration_q(True)))